# Equipo 2
## Miembros:
### Jorge Arturo Torres Cruz - A01176590

## Recolección de datos

In [0]:
url_files = ['wines.txt', 'bubble_gums.txt', 'dumplings.txt', 'pizza_urls.txt', 'sandwich.txt']

In [11]:
# Iteramos por cada uno de los archivos y obtenemos los URLs de las imagenes.
print("Reading files containing images urls")
urls = []
for file_name in url_files:
  with open(file_name) as f:
    content = f.readlines()
    content = [url.strip() for url in content]
    print("Reading {} image urls ({})".format(len(content), file_name))
    urls.append(content)

Reading files containing images urls
Reading 1248 image urls (wines.txt)
Reading 1210 image urls (bubble_gums.txt)
Reading 1273 image urls (dumplings.txt)
Reading 1216 image urls (pizza_urls.txt)
Reading 1230 image urls (sandwich.txt)


In [0]:
# Utilizaremos urllib para descargar las imagenes utilizando URLs obtenidos de ImageNet
import urllib.request
import os

In [9]:
categories = ['wine', 'bubble_gum', 'dumplings', 'pizza', 'sandwich']
os.mkdir("./data")
for idx, category in enumerate(categories):
  counter = 0
  try:
    print("Creating directory to store {} images".format(category))
    os.mkdir("./data/{}".format(category))
  except Exception as e:
    print(e)
  else:
    print("Succesfully created {}/ directory".format(category))
  for img in range(0, len(urls[idx])):
    try:
      urllib.request.urlretrieve(urls[idx][img], "{}/{}.jpg".format(category, counter))
      counter += 1
    except Exception as e:
      print("==>An error ocurred while fetching img #{} from {}".format(img, category))
      print(e)

Creating directory to store wine images
Succesfully created wine/ directory


NameError: ignored